# **UNIVERSIDAD TORCUATO DI TELLA**
## **MAESTRÍA EN ECONOMETRÍA**

---

### **TRABAJO PRÁCTICO DE ECONOMETRÍA**

- **Profesor:** González-Rozada, Martín  
- **Ayudante:** Lening, Iara  
- **Alumno:** Guzzi, David Alexander  (Legajo n°: 24H1970)  

**Ciclo Lectivo:** Tercer Trimestre, 2024  

---

##### **1. PROPIEDADES DE MUESTRA FINITA DE FGLS (MCGE).**

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Parámetros del modelo
beta_0 = -3
beta_1 = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de covarianza
n_observations = 5  # Tamaño de cada muestra
n_samples = 5000  # Número de muestras


# Generación de muestras
samples = []
np.random.seed(3649)  #Últimos 4 números de mi documento de identidad.

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1 * x + u  # Generar y
    samples.append(pd.DataFrame({'x': x, 'u': u, 'y': y}))

# Combinar todas las muestras en un único DataFrame
all_samples = pd.concat(samples, keys=range(n_samples), names=['sample', 'index']).reset_index()

# Mostrar algunas filas
all_samples.head(10)

,sample,index,x,u,y
0,0,0,44.691948,0.394367,33.147926
1,0,1,33.945501,-2.739349,21.417052
2,0,2,37.364044,-5.117829,21.773406
3,0,3,37.251050,1.502358,28.303198
4,0,4,34.525174,6.038023,30.658162
5,1,0,3.409759,0.030947,-0.241246
6,1,1,14.352028,-0.504023,7.977600
7,1,2,30.112575,2.001504,23.091564
8,1,3,5.797610,3.095406,4.733495
9,1,4,20.579967,8.232626,21.696599


In [5]:
import numpy as np
import pandas as pd
from scipy.stats import t

# Parámetros iniciales
beta_0 = -3
beta_1_true = 0.8
omega = np.diag([4, 9, 16, 25, 36])  # Matriz de varianzas
n_samples = 5000
n_observations = 5
alpha_levels = [0.01, 0.05]  # Niveles de significancia

# Función para estimar por FGLS
def fgls_estimation(x, y, omega):
    W = np.linalg.inv(omega)  # Matriz de pesos inversos
    X = np.column_stack((np.ones(len(x)), x))  # Matriz de diseño
    beta_hat = np.linalg.inv(X.T @ W @ X) @ (X.T @ W @ y)  # Estimación FGLS
    var_beta = np.linalg.inv(X.T @ W @ X)  # Varianza de los estimadores
    return beta_hat, np.sqrt(np.diag(var_beta))  # Estimadores y errores estándar

# Simulaciones
results = []
np.random.seed(42)

for _ in range(n_samples):
    x = np.random.uniform(1, 50, n_observations)  # x ~ U[1, 50]
    u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)  # u ~ N(0, omega)
    y = beta_0 + beta_1_true * x + u  # Generar y
    beta_hat, se_beta = fgls_estimation(x, y, omega)  # Estimación FGLS
    t_stat = (beta_hat[1] - beta_1_true) / se_beta[1]  # Estadístico t para beta_1
    p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_observations - 2))  # Valor p
    results.append([beta_hat[0], beta_hat[1], se_beta[0], se_beta[1], t_stat, p_value])

# Convertir resultados a DataFrame
results_df = pd.DataFrame(results, columns=['beta_0', 'beta_1', 'se_beta_0', 'se_beta_1', 't_stat', 'p_value'])

# Tamaño del test para diferentes niveles de significancia
size_test = {
    alpha: (results_df['p_value'] < alpha).mean() for alpha in alpha_levels
}

# Poder del test para diferentes valores alternativos de beta_1
power_results = {}
for beta_1_alt in [0, 0.4]:
    power = []
    for _ in range(n_samples):
        x = np.random.uniform(1, 50, n_observations)
        u = np.random.multivariate_normal(mean=np.zeros(n_observations), cov=omega)
        y = beta_0 + beta_1_alt * x + u
        beta_hat, se_beta = fgls_estimation(x, y, omega)
        t_stat = (beta_hat[1] - beta_1_true) / se_beta[1]
        p_value = 2 * (1 - t.cdf(abs(t_stat), df=n_observations - 2))
        power.append(p_value < 0.05)
    power_results[beta_1_alt] = np.mean(power)

# Estadísticas descriptivas
descriptive_stats = results_df[['beta_0', 'beta_1']].agg(['mean', 'median', 'std'])

# Resultados finales
print("Tamaño del test:", size_test)
print("Poder del test:", power_results)
print("Estadísticas descriptivas:\n", descriptive_stats)

Tamaño del test: {0.01: 0.0, 0.05: 0.0018}
Poder del test: {0: 0.907, 0.4: 0.4572}
Estadísticas descriptivas:
           beta_0    beta_1
mean   -3.124395  0.803626
median -3.050941  0.804804
std     4.543793  0.162838
